# Test Language Model

In [1]:
import torch
import models
import utils_char_lm
import utils_char_dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('Device:', device)

model_path = './model_lm_best.pt'

# The codemap is a dictionary of words to class index
codemap = utils_char_dataset.load_pickle('./codemap_LM.pickle')

num_classes = len(codemap)
print('Num classes:', num_classes)

# Hyperparameters
# Pure sequence to sequence models can't deal with batches
hidden_size = 256
num_layers = 2

Device: cuda:0
Num classes: 69


#### Load Model Checkpoint

In [2]:
char_lm = models.CharLangModel(num_classes, hidden_size, num_classes, num_layers=num_layers)
char_lm = char_lm.to(device)

checkpoint = torch.load(model_path)
char_lm.load_state_dict(checkpoint['seq_model']);
char_lm.eval();

#### Get Probabilities of sequence of Words

In [33]:
lst_words = ['Thi1', 'This', 'Love', 'BrA']
for word in lst_words:
    print('P(%s):%f' % (word, utils_char_lm.getProbabilitySentence(word, char_lm, device, codemap)))

P(Thi1):0.000000
P(This):0.052008
P(Love):0.021699
P(BrA):0.000013


In [9]:
utils_char_lm.getProbabilitySentence('Hello', char_lm, device, codemap) > utils_char_lm.getProbabilitySentence('Hell0', char_lm, device, codemap)

True

In [50]:
import numpy as np
def prob_sentence(word, model, device, codemap):        
    # Convert each character on the word into it's class id
    chars_class = [utils_char_dataset.class_id_from_char(char, codemap) for char in word]
    print('chars_class:', chars_class)
    num_chars = len(chars_class)   
    print('num_chars:', num_chars)
    curr_batch_size = 1            
    model.eval()
    scores_lst = []
    with torch.no_grad():
        # Initialize model on the beginning of the sequence
        hidden_state = models.initHidden(curr_batch_size, False, model.hidden_size, model.num_layers, device)
        # Iterate on all charactres from word ie: Hello --> [23, 46, 53, 53, 56]
        for idx in range(num_chars):
            # Convert class word index to a tensor
            input = torch.tensor(chars_class[idx]).type(torch.LongTensor).unsqueeze(0).unsqueeze(0).to(device)   
            # Push input(character) to the model
            # Probabilities shape [1 x 1 x num_classes]
            probabilities, hidden_state = model(input, hidden_state, torch.tensor(1).unsqueeze(0))
            print('probabilities:', probabilities.shape)
            # Get the probability of the next input character given the previous inputs 
            if idx < num_chars - 1:
                print('\tidx:', idx)
                input_next = torch.tensor(chars_class[idx+1]).type(torch.LongTensor).unsqueeze(0).unsqueeze(0).to(device).item()                
                probabilities = probabilities.squeeze(0).squeeze(0)    
                prob_next = probabilities[input_next].item()
                scores_lst.append(prob_next)                   
            
    # Return the product of the probabilities
    return np.prod(scores_lst)

In [51]:
prob_sentence('Hello', char_lm, device, codemap)

chars_class: [23, 46, 53, 53, 56]
num_chars: 5
probabilities: torch.Size([1, 1, 69])
	idx: 0
probabilities: torch.Size([1, 1, 69])
	idx: 1
probabilities: torch.Size([1, 1, 69])
	idx: 2
probabilities: torch.Size([1, 1, 69])
	idx: 3
probabilities: torch.Size([1, 1, 69])


2.0186739064190997e-07

#### Generate Sequences

In [26]:
lst_words = ['t', 'k', 'de', 'Liv', 'Pre', 'To be or not to b']
for word in lst_words:
    pred = utils_char_lm.getNextChar(word,100, char_lm, device, codemap, greedly=False)
    res_str = ''.join([utils_char_dataset.char_from_class_id(class_id, codemap) for class_id in pred])
    print('('+word+')'+res_str)

(t)he<EOS>
(k)now<EOS>
(de)spise<EOS>
(Liv)ed<EOS>
(Pre)ferr'd,<EOS>
(To be or not to b).<EOS>
